#Module 4: ANOVA

##Blood clotting
Investigators are measuring how regular use of a commonly used painkiller affects blood clotting in patients over time, assuming a significance of 0.05. The experiment was designed as follows:


*   Eight subjects were gathered. Each had their blood tested for clotting time (minutes) before use of the painkiller as a control.
*   Subjects began regular use of the painkiller. Clotting time was then measured three months after drug use began.
*   Clotting time was then measured six months after drug use began.

Before we dive into the Python, discuss briefly:


*   Would a regular or repeated measures ANOVA be appropriate to assess the drug's effect on clotting time?
*   Propose a null and alternate hypothesis for this study.
*   List the assumptions that must be valid (or close to valid) when drawing conclusions from your test.


In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/jlongc12/stats_py/main/data/clot_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   subject  8 non-null      int64
 1   control  8 non-null      int64
 2   3mo      8 non-null      int64
 3   6mo      8 non-null      int64
dtypes: int64(4)
memory usage: 384.0 bytes


## Calculating sum of squares
Fill in the table below with calculated values.

| | Sum of squares (SS) | Degrees of freedom (DF) |
| --- | --- | --- |
| Total | | |
| Between groups | | |
| Within groups | | |
| Subjects groups | | |
| Error | | |


In [2]:
# A lot of operations here. Time to practice your Python skills!
total = np.array(df[['control','3mo','6mo']])
grand_mean = total.mean()

ss_total = np.sum(np.square(total-grand_mean))

ss_control = np.square(df['control'].mean()-grand_mean)
ss_3mo = np.square(df['3mo'].mean()-grand_mean)
ss_6mo = np.square(df['6mo'].mean()-grand_mean)
ss_bg = (ss_control+ss_3mo+ss_6mo)*df['subject'].count()

ss_control = np.sum(np.square(df['control']-df['control'].mean()))
ss_3mo = np.sum(np.square(df['3mo']-df['3mo'].mean()))
ss_6mo = np.sum(np.square(df['6mo']-df['6mo'].mean()))
ss_wg = ss_control+ss_3mo+ss_6mo

ss_subject = np.sum(np.square(total.mean(axis=1)-grand_mean))*np.size(total,axis=1)
ss_error = ss_wg-ss_subject


print('Sum of squares:')
print('Total: %.2f' % ss_total)
print('Between group: %.2f' % ss_bg)
print('Within group: %.2f' % ss_wg)
print('Subject: %.2f' % ss_subject)
print('Error: %.2f' % ss_error)

Sum of squares:
Total: 194.00
Between group: 82.75
Within group: 111.25
Subject: 62.00
Error: 49.25


## Calculating the F-statistic
Fill in the table below with calculated values. When comparing to a significance of 0.05, what can you conclude about the effect of painkillers on mean clotting time over time?

| | Mean sum of squares (MSS) | F-statistic | p-value |
| --- | --- | --- | --- |
| Between group | | | |
| Error | | N/A | N/A |


In [ ]:
df_bg = np.size(total,axis=1)-1
mss_bg = ss_bg/df_bg

df_wg = np.prod(np.shape(total))-1-df_bg
df_subject = np.size(total,axis=0)-1
df_error = df_wg-df_subject
mss_error = ss_error/df_error

f_stat = mss_bg/mss_error

# Similar to a function we've seen before. Check the docs!
p = 1-stats.f.cdf(f_stat,df_bg,df_error)

print('Mean sum of squares:')
print('Between group: %.2f' % mss_bg)
print('Error: %.2f' % ss_error)
print('F-statistic: %.2f' % f_stat)
print('p: %.3f' % p)

Mean sum of squares:
Between group: 41.38
Error: 49.25
F-statistic: 11.76
p: 0.001


F_onewayResult(statistic=7.810112359550562, pvalue=0.002912272720024801)

## Testing assumptions

Let's now test the assumptions that we assumed were valid at the start. Test for the following assumptions: normality and homogeneity of variance. What do the results indicate?

In [ ]:
# You can probably guess what I'd say here
p_shapiro = stats.shapiro(total)
p_levene = stats.levene(total[:,0],total[:,1],total[:,2])

print('p value for Shapiro-Wilk: %.2f' % p_shapiro[1])
print('p value for Levene: %.2f' % p_levene[1])

p value for Shapiro-Wilk: 0.59
p value for Levene: 0.28


## Post hoc testing
Let's go back and run some post hoc tests on the data to determine which groups are significantly different than one another. Perform a Tukey test and report your findings.

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Some moving the data frame around to prep it for use.
df_tukey = pd.DataFrame({'clot_time': np.concatenate([np.array(df['control']),
                                                      np.array(df['3mo']),
                                                      np.array(df['6mo'])]),
                         'group': np.concatenate([np.repeat('control',repeats=df['subject'].count()),
                                                  np.repeat('3mo',repeats=df['subject'].count()),
                                                  np.repeat('6mo',repeats=df['subject'].count())])})

tukey = pairwise_tukeyhsd(endog=df_tukey['clot_time'],
                          groups=df_tukey['group'],
                          alpha=0.05)

print(tukey)



Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1  group2 meandiff p-adj   lower  upper  reject
----------------------------------------------------
   3mo     6mo   -0.125    0.9 -3.0249 2.7749  False
   3mo control    3.875 0.0078  0.9751 6.7749   True
   6mo control      4.0 0.0061  1.1001 6.8999   True
----------------------------------------------------
